In [1]:
from agent import QLearningAgent
from verify_data import FishGame, ParseError

In [2]:
import os

memories = []
for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    with open(filepath, 'r') as f:
        try:
            print(f"{filename}")
            game = FishGame(f.readlines())
            for player in game.players:
                memories += game.memory(player)
        except ParseError as e:
            print(f"{filename}: {e}")
            break

12-3_15:27.txt
12-10_11:07.txt
12-3_14:05.txt
12-3_11:30.txt
12-3_14:27.txt
12-6_11:08.txt
12-3_11:12.txt
12-4_11:11.txt
1-15_11:15.txt


In [3]:
agent = QLearningAgent()
agent.train_on_data(memories, 100)

/Users/Yourui/Documents/Fish/.env/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Users/Yourui/Documents/Fish/agent.py:72: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  return torch.FloatTensor(x).to(self.device)
/Users/Yourui/Documents/Fish/agent.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'ask_card': F.softmax(ask_card),
/Users/Yourui/Documents/Fish/agent.py:93: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to bro

epoch 0, loss 4.05939, lr 0.001
epoch 1, loss 3.27221, lr 0.001
epoch 2, loss 3.06853, lr 0.001
epoch 3, loss 3.09898, lr 0.001
epoch 4, loss 3.57741, lr 0.001
epoch 5, loss 2.87866, lr 0.001
epoch 6, loss 2.62066, lr 0.001
epoch 7, loss 2.23506, lr 0.001
epoch 8, loss 2.15655, lr 0.001
epoch 9, loss 2.06953, lr 0.001
epoch 10, loss 1.94519, lr 0.001
epoch 11, loss 1.98701, lr 0.001
epoch 12, loss 1.99226, lr 0.001
epoch 13, loss 2.03960, lr 0.001
epoch 14, loss 1.98626, lr 0.001
epoch 15, loss 1.97667, lr 0.001
epoch 16, loss 1.90515, lr 0.001
epoch 17, loss 1.93229, lr 0.001
epoch 18, loss 1.91518, lr 0.001
epoch 19, loss 1.96292, lr 0.001
epoch 20, loss 1.90613, lr 0.001
epoch 21, loss 1.89529, lr 0.001
epoch 22, loss 1.88928, lr 0.001
epoch 23, loss 1.90747, lr 0.001
epoch 24, loss 1.93551, lr 0.001
epoch 25, loss 1.84701, lr 0.001
epoch 26, loss 1.87153, lr 0.001
epoch 27, loss 1.87701, lr 0.001
epoch 28, loss 1.88308, lr 0.001
epoch 29, loss 1.85960, lr 0.001
epoch 30, loss 1.883

In [4]:
import os
from datetime import datetime

os.makedirs('models', exist_ok=True)
model_path = f'models/fish_agent_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth'
agent.save_model(model_path)

Model saved to models/fish_agent_20250324_150719.pth


In [7]:
agent = QLearningAgent() 
agent.load_model('models/fish_agent_20250319_123306.pth')

True

In [8]:
game = FishGame(open('data/1-15_11:15.txt', 'r').readlines())
player = 'PB'
test_data = agent.unpack_batch(game.memory(player)[1:2])
agent.q_network(agent.tensor(test_data['state']), 
                agent.action_masks(*test_data['mask_dep'].values()))

{'call': tensor([[0.3735, 0.6265]], device='mps:0', grad_fn=<SoftmaxBackward0>),
 'call_set': tensor([[0.0683, 0.3328, 0.1711, 0.0416, 0.1856, 0.0393, 0.1124, 0.0488, 0.0000]],
        device='mps:0', grad_fn=<SoftmaxBackward0>),
 'call_cards': tensor([[[0.2083, 0.2714, 0.2998, 0.2205],
          [0.1354, 0.2199, 0.3274, 0.3173],
          [0.4047, 0.3686, 0.0762, 0.1506],
          [0.1540, 0.3419, 0.0747, 0.4293],
          [0.3549, 0.1006, 0.3225, 0.2219],
          [0.1586, 0.2986, 0.3635, 0.1793]]], device='mps:0',
        grad_fn=<SoftmaxBackward0>),
 'ask_person': tensor([[0.2090, 0.1333, 0.6202, 0.0376]], device='mps:0',
        grad_fn=<SoftmaxBackward0>),
 'ask_set': tensor([[0.0000, 0.1338, 0.0000, 0.6462, 0.0475, 0.1345, 0.0380, 0.0000, 0.0000]],
        device='mps:0', grad_fn=<SoftmaxBackward0>),
 'ask_card': tensor([[2.7193e-01, 9.0935e-02, 6.2667e-01, 3.1250e-06, 1.6242e-06, 1.0456e-02]],
        device='mps:0', grad_fn=<SoftmaxBackward0>)}